In [1]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

2024-02-14 22:52:54.322035: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-02-14 22:52:54.452542: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-14 22:52:54.452582: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-14 22:52:54.472727: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-02-14 22:52:54.513272: I tensorflow/core/platform/cpu_feature_guar

In [7]:


class grad_initilization:
    [tf.config.experimental.set_memory_growth(device, True) for device in tf.config.list_physical_devices('GPU')]
    physical_devices = tf.config.list_physical_devices('GPU')
    if physical_devices:
        tf.config.experimental.set_memory_growth(physical_devices[0], True)
    else:
        raise RuntimeError("No GPU devices found.")

    
    def __init__(self, loss_function='mse', physical_devices=0):
        self.loss_function = loss_function
        self.initial_prediction = None
        self.physical_devices = physical_devices
        
    ### Find a good starting point for point. ie. initializing the gradient boosting alo
    def target_mean(self, y):
        return np.mean(y)

    ### MSE = The squared error between the predicted value and the actual value
    ### So when the algo guess guesses y for a given x, this measures how far off p is from y
    def mse(self, y, p):
        return np.mean((y - p)**2) 

    def startpoint_mse(self, y):
        """Initialize the gradient boosting model."""
        if self.loss_function == 'mse':
            self.initial_prediction = self.target_mean(y)
            print(f"Initial model prediction (mean of y): {self.initial_prediction}")
            initial_mse = self.mse(y, self.initial_prediction)
            print(f"Initial MSE: {initial_mse}")
            return self.initial_prediction
        else:
            raise ValueError("Unsupported loss function. Currently, only 'mse' is implemented.")
    
    def mse(self, y, p):
        if self.loss_function != 'mse':
            raise ValueError("Unsupported loss function. Currently, only 'mse' is implemented.")
        else:
            return np.mean((y - p)**2)
    
    def learner(self, X, residuals, max_depth=3, loss_function='mse', physical_devices=0, **kwargs):
        """Fit a decision tree to the residuals."""
        if self.loss_function != 'mse':
            raise ValueError("Unsupported loss function. Currently, only 'mse' is implemented.")
        else:
            tree = DecisionTreeRegressor(max_depth=self.max_depth)
            tree.fit(X, residuals)
            return tree
            


if __name__ == "__main__":
    n = 4096
    y = tf.random.uniform((n, n), dtype=float)
    
    # Initialize the gradient boosting model
    gbm_initializer = grad_initilization()
    gbm_initializer.startpoint_mse(y)

Initial model prediction (mean of y): 0.5000036954879761
Initial MSE: 0.08332838118076324


In [5]:
def mse(self, y, p):
    if self.loss_function != 'mse':
        raise ValueError("Unsupported loss function. Currently, only 'mse' is implemented.")
    else:
        return np.mean((y - p)**2)

In [6]:
import numpy as np

def best_split(X, y):
    """Find the best split for a node."""
    m, n = X.shape
    if m <= 1:
        return None, None
    
    # Current variance
    best_mse = np.var(y)
    best_idx, best_val = None, None
    
    for idx in range(n):  # for each feature
        thresholds = np.unique(X[:, idx])
        for t in thresholds:
            left_idx = X[:, idx] < t
            right_idx = X[:, idx] >= t
            if np.sum(left_idx) == 0 or np.sum(right_idx) == 0:
                continue
            
            left_mse = np.var(y[left_idx])
            right_mse = np.var(y[right_idx])
            mse = (np.sum(left_idx) * left_mse + np.sum(right_idx) * right_mse) / m
            
            if mse < best_mse:
                best_mse = mse
                best_idx = idx
                best_val = t
                
    return best_idx, best_val


In [ ]:
class TreeNode:
    def __init__(self, X, y, idxs, depth=0, max_depth=3):
        self.X = X
        self.y = y
        self.idxs = idxs
        self.depth = depth
        self.max_depth = max_depth
        self.left = None
        self.right = None
        self.is_leaf = False
        self.val = None
        self.split_idx = None
        self.split_val = None
        
        self._split()
    
    def _split(self):
        if self.depth == self.max_depth or len(self.idxs) <= 1:
            self.is_leaf = True
            self.val = np.mean(self.y[self.idxs])
            return
        
        split_idx, split_val = best_split(self.X[self.idxs], self.y[self.idxs])
        if split_idx is None:
            self.is_leaf = True
            self.val = np.mean(self.y[self.idxs])
            return
        
        self.split_idx = split_idx
        self.split_val = split_val
        
        left_idxs = np.nonzero(self.X[self.idxs, split_idx] < split_val)[0]
        right_idxs = np.nonzero(self.X[self.idxs, split_idx] >= split_val)[0]
        
        self.left = TreeNode(self.X, self.y, self.idxs[left_idxs], self.depth+1, self.max_depth)
        self.right = TreeNode(self.X, self.y, self.idxs[right_idxs], self.depth+1, self.max_depth)
    
    def predict(self, x):
        if self.is_leaf:
            return self.val
        elif x[self.split_idx] < self.split_val:
            return self.left.predict(x)
        else:
            return self.right.predict(x)


In [ ]:
class DecisionTreeRegressor:
    def __init__(self, max_depth=3):
        self.max_depth = max_depth
        self.root = None
    
    def fit(self, X, y):
        self.root = TreeNode(X, y, np.arange(len(y)), max_depth=self.max_depth)
    
    def predict(self, X):
        return np.array([self.root.predict(x) for x in X])


In [ ]:
def learner(self, X, residuals, max_depth=3, loss_function='mse'):
    """Fit a decision tree to the residuals."""
    if loss_function != 'mse':
        raise ValueError("Unsupported loss function. Currently, only 'mse' is implemented.")
    
    tree = DecisionTreeRegressor(max_depth=max_depth)
    tree.fit(X, residuals)
    return tree
